#   实践课题报告： 大数据文本分析与应用
    
   #### 学       校：西南林业大学
   #### 学       院：大数据与智能工程学院
   #### 专       业：信息工程（数据科学与大数据技术）
   #### 年       级：2017级
   #### 学       号：20171257028
   #### 姓       名：张开富
   #### 指导老师：赵友杰
   #### 日       期：2019 年 6 月 27 日 至  29 日

# 实习要求：
        使用Python，实现对英文小说的词频解析，并将每章的词频结果存入到Mysql数据库中。
    开发简易信息系统，实现对某篇小说的词频查询。

# 第一步：读取数据，并进行预处理 

In [1]:
#获取数据并对其进行去空行,空白处理
def get_data( filename):
    data = ''
    with open(filename, "r") as f:
        for line in f.readlines():
            line = line.rstrip() #去掉列表右边的空白
            line = line.lstrip() #去掉列表左边的空白
            #全变为小写
            #line = line.lower()
            #将特殊字符替换为空格
            for ch in '!"#$%@~`^&*\'\“”’。，()_-+=/{\\}[|]?.><,/;':
                line = line.replace(ch, " ") 
            if line is not '':
                data = data+line+' '
    return data

In [70]:
# txt文件和当前脚本在同一目录下，所以不用写具体路径
filename = 'Two_Cities.txt'

In [71]:
data=get_data(filename)

# 第二步：按章节进行分词和统计

In [78]:
#获取章节单词 ,key="CHAPTER",章节分割词
def get_chapter_word(key ,data):
    #分词
    words = data.split()
    chapters ={}
    j=0
    for word in words:
        if(word == key):
                j+=1
        if j == 0:
            chapters['book'] = chapters.get('book','')+word+" "
        else:
            keys = key+str(j)
            chapters[keys] = chapters.get(keys,'')+word+" "
    chapter_words = list(chapters.items())
    return chapter_words

In [5]:
#按按章节进行词频统计
def word_count(chapter_word):
     #全变为小写
    chapter_word = chapter_word.lower()
    words = chapter_word.split()
    #字典存储单词和数量
    counts ={}
    for word in words:
        counts[word] = counts.get(word,0) + 1
    items = list(counts.items())
    #大到小快速排序
    items.sort(key=lambda x:x[1], reverse=True)
    return items

# 第三步：数据库操作

In [128]:
import pymysql
#连接数据库
def connectdb():
    localhost = " localhost"
    username = "root"
    password = "123456"
    DBname = "test"
    #print('连接到mysql服务器...')
    # 打开数据库连接
    db = pymysql.connect(localhost,username,password,DBname)
    #print('连接上了!')
    return db
    
#存入数据库,书名，章节，单词，词频
def insert(db,bookname,chapter,word, counts):
    # 使用cursor()方法获取操作游标 
    cursor = db.cursor()
    # SQL 插入语句
    sql = "INSERT INTO txtdata(bookname, \
       chapter, word, counts) \
       VALUES ('%s', '%s', '%s',  %s)" % \
       (bookname, chapter,word, counts)
    try:
        # 执行sql语句
        cursor.execute(sql)
        # 执行sql语句
        db.commit()
    except:
        # 发生错误时回滚
        db.rollback()

#查询数据库
def select(sql):
    #连接数据库
    db= connectdb()
    # 使用cursor()方法获取操作游标 
    cursor = db.cursor()
    try:
        #记录总的次数
        count = 0
        # 执行SQL语句
        cursor.execute(sql)
        # 获取所有记录列表
        results = cursor.fetchall()
        for row in results:
            bookname = row[0]
            chapter = row[1]
            counts = row[2]
            count += counts
            # 打印结果
            print ("书名:%s \t 章节：%s \t 词频：%d" %(bookname,chapter,counts))
        print("总的出现频率为：%d"%(count))
    except:
        print ("词库无此单词！！！")
    # 关闭数据库连接
    db.close()  
    
#按单词查询所有的词频记录
def select_by_word(word):
    # SQL 查询语句
    sql = "SELECT bookname, chapter,counts FROM txtdata WHERE trim(replace(word,' ','')) = trim(replace('%s',' ',''))" % (word)
    #调用查询函数
    select(sql)
    
#查询指定书的查询指定单词的词频
def select_by_bookname(bookname,word):
    # SQL 查询语句
    sql = "SELECT bookname, chapter,counts FROM txtdata WHERE trim(replace(bookname,' ','')) =\
    trim(replace('%s',' ','')) and trim(replace(word,' ','')) = trim(replace('%s',' ',''))" % (bookname,word)
    #调用查询函数
    select(sql)

# 第四步：存储数据，查询词频

In [82]:
#存入数据库主函数
def insert_main(filename):
    #filename = 'Jane_Eyre.txt'
    #读取数据
    data=get_data(filename)
    #获取章节单词 ,key="CHAPTER",章节分割词库
    key="CHAPTER"
    #录入信息条数
    count = 0
    
     #连接数据库
    db= connectdb()

    chapter_words = get_chapter_word(key ,data)
    for i in range(len(chapter_words)):
        if i == 0:
            continue
        else:
            chapter,chapter_word = chapter_words[i]
            chapter_word = chapter_word.rstrip().lstrip()
            #按按章节进行词频统计
            items = word_count(chapter_word)
            for j in range(len(items)):
                word, counts= items[j]
                #print("{0:<10}{1:>5}".format(word, count))
                try:
                    #存入数据库,书名，章节，单词，词频
                    insert(db,chapter_words[0][1],chapter,word, counts)
                except:
                    continue
                count+=1
        print("已录入%d章节的数据》》》"%(i))
    # 关闭数据库连接
    db.close()  
    print("本次总共录入%d条数据"%(count))

In [60]:
#按单词查询主函数
def select_main():
    word = input("请输入你要查询的单词：")
    word = word.lower()
    print("\n查询结果：\n")
    select_by_word(word)

In [67]:
##查询指定书的查询指定单词的词频主函数
def select_by_bookname_main():
    bookname = input("请输入书名：")
    word = input("\n请输入你要查询的单词：")
    word = word.lower()
    print("\n查询结果：\n")
    select_by_bookname(bookname,word)

# 第五步：测试

## 存入数据库

In [45]:
# txt文件和当前脚本在同一目录下，所以不用写具体路径
filename_path = ['Two_Cities.txt','Jane_Eyre.txt']
for filename in filename_path:
    #存入数据库
   # insert_main(filename)

已录入1章节的数据》》》
已录入2章节的数据》》》
已录入3章节的数据》》》
已录入4章节的数据》》》
已录入5章节的数据》》》
已录入6章节的数据》》》
已录入7章节的数据》》》
已录入8章节的数据》》》
已录入9章节的数据》》》
已录入10章节的数据》》》
已录入11章节的数据》》》
已录入12章节的数据》》》
已录入13章节的数据》》》
已录入14章节的数据》》》
已录入15章节的数据》》》
已录入16章节的数据》》》
已录入17章节的数据》》》
已录入18章节的数据》》》
已录入19章节的数据》》》
已录入20章节的数据》》》
已录入21章节的数据》》》
已录入22章节的数据》》》
已录入23章节的数据》》》
已录入24章节的数据》》》
已录入25章节的数据》》》
已录入26章节的数据》》》
已录入27章节的数据》》》
已录入28章节的数据》》》
已录入29章节的数据》》》
已录入30章节的数据》》》
已录入31章节的数据》》》
已录入32章节的数据》》》
已录入33章节的数据》》》
已录入34章节的数据》》》
已录入35章节的数据》》》
已录入36章节的数据》》》
已录入37章节的数据》》》
已录入38章节的数据》》》
已录入39章节的数据》》》
已录入40章节的数据》》》
已录入41章节的数据》》》
已录入42章节的数据》》》
已录入43章节的数据》》》
已录入44章节的数据》》》
本次总共录入%d条数据
已录入1章节的数据》》》
已录入2章节的数据》》》
已录入3章节的数据》》》
已录入4章节的数据》》》
已录入5章节的数据》》》
已录入6章节的数据》》》
已录入7章节的数据》》》
已录入8章节的数据》》》
已录入9章节的数据》》》
已录入10章节的数据》》》
已录入11章节的数据》》》
已录入12章节的数据》》》
已录入13章节的数据》》》
本次总共录入%d条数据


## 按单词查询所有

In [123]:
 #查询1
select_main()

请输入你要查询的单词：the

查询结果：

书名: Two Cities 	 章节：CHAPTER1 	 词频：79
书名: Two Cities 	 章节：CHAPTER2 	 词频：194
书名: Two Cities 	 章节：CHAPTER3 	 词频：123
书名: Two Cities 	 章节：CHAPTER4 	 词频：203
书名: Two Cities 	 章节：CHAPTER5 	 词频：284
书名: Two Cities 	 章节：CHAPTER6 	 词频：331
书名: Two Cities 	 章节：CHAPTER7 	 词频：171
书名: Two Cities 	 章节：CHAPTER8 	 词频：298
书名: Two Cities 	 章节：CHAPTER9 	 词频：89
书名: Two Cities 	 章节：CHAPTER10 	 词频：137
书名: Two Cities 	 章节：CHAPTER11 	 词频：228
书名: Two Cities 	 章节：CHAPTER12 	 词频：263
书名: Two Cities 	 章节：CHAPTER13 	 词频：166
书名: Two Cities 	 章节：CHAPTER14 	 词频：348
书名: Two Cities 	 章节：CHAPTER15 	 词频：94
书名: Two Cities 	 章节：CHAPTER16 	 词频：41
书名: Two Cities 	 章节：CHAPTER17 	 词频：102
书名: Two Cities 	 章节：CHAPTER18 	 词频：56
书名: Two Cities 	 章节：CHAPTER19 	 词频：212
书名: Two Cities 	 章节：CHAPTER20 	 词频：87
书名: Two Cities 	 章节：CHAPTER21 	 词频：284
书名: Two Cities 	 章节：CHAPTER22 	 词频：208
书名: Two Cities 	 章节：CHAPTER23 	 词频：118
书名: Two Cities 	 章节：CHAPTER24 	 词频：152
书名: Two Cities 	 章节：CHAPTER25 	 词频：34
书名: Two Cities 	 章

## 按指定书名查询指定单词的词频

In [129]:
 #查询2
select_by_bookname_main()

请输入书名：Jane Eyre

请输入你要查询的单词：the

查询结果：

书名: Jane Eyre 	 章节：CHAPTER1 	 词频：86
书名: Jane Eyre 	 章节：CHAPTER2 	 词频：104
书名: Jane Eyre 	 章节：CHAPTER3 	 词频：118
书名: Jane Eyre 	 章节：CHAPTER4 	 词频：203
书名: Jane Eyre 	 章节：CHAPTER5 	 词频：264
书名: Jane Eyre 	 章节：CHAPTER6 	 词频：110
书名: Jane Eyre 	 章节：CHAPTER7 	 词频：178
书名: Jane Eyre 	 章节：CHAPTER8 	 词频：90
书名: Jane Eyre 	 章节：CHAPTER9 	 词频：128
书名: Jane Eyre 	 章节：CHAPTER10 	 词频：179
书名: Jane Eyre 	 章节：CHAPTER11 	 词频：250
书名: Jane Eyre 	 章节：CHAPTER12 	 词频：164
书名: Jane Eyre 	 章节：CHAPTER13 	 词频：160
总的出现频率为：2034
